# NOTEBOOK FOR DETECTION OF HAND GESTURE 

In [1]:
#Importing necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
import cv2
import numpy as np
from skimage.transform import resize,pyramid_reduce

Refer my ASL_datacreation notebook for understanding of each and every line in code

In [2]:
import time

In [3]:
model1=load_model(r'ASL_CNN_MODEL1.h5') #Loading the model

In [4]:
background = None
accumulated_weight=0.5
def cal_accum_avg(frame,accumulated_weight):
    global background
    if(background is None):
        #background=frame.copy().astype('float')
        background=frame.copy().astype('float')
        return(None)
    
    cv2.accumulateWeighted(frame,background,accumulated_weight)
        
    

In [5]:
def hand_segmentation(frame,threshold=25):
    global background
    diff=cv2.absdiff(background.astype('uint8'),frame)
    
    _ , thresholded=cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY)
    
    contours,hei=cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    
    
    if(len(contours)==0):
        return(None)
    
    else:
        hand_segment_max=max(contours,key=cv2.contourArea)
        
        return(thresholded,hand_segment_max)
        
    
    
    
    

In [6]:
word_dict={
    0: "A",
    1: "B",
    2: "C",
    3: "D",
    4: "E",
    5: "F",
    6: "G",
    7: "H",
    8: "I",
    9: "J",
    10: "K",
    11: "L",
    12: "M",
    13: "N",
    14: "O",
    15: "P",
    16: "Q",
    17: "R",
    18: "S",
    19: "T",
    20: "U",
    21: "V",
    22: "W",
    23: "X",
    24: "Y",
    25: "Z",
    26: "background"
}

In [7]:
ROI_top = 100
ROI_bottom = 300
ROI_right = 150
ROI_left = 350

camera=cv2.VideoCapture(0)
time.sleep(2) 
num_frames=0
while True:
    _,image_frame=camera.read()
    
    image_frame=cv2.flip(image_frame,1)
    
    img=image_frame.copy()
    
    roi=image_frame[ROI_top:ROI_bottom,ROI_right:ROI_left]
    
    
    img_grayscale=cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    
    img_grayscale_blurr=cv2.GaussianBlur(img_grayscale,(9,9),0)
    
    if(num_frames<70):
        
        cal_accum_avg(img_grayscale_blurr,accumulated_weight)
        
        cv2.putText(img,"FETCHING BACKGROUND PLEASE WAIT",(80,350),cv2.FONT_HERSHEY_SIMPLEX,
                    0.9,(0,0,255),2)
        
    else:
        hand=hand_segmentation(img_grayscale_blurr)
        
        
        if(hand is not None):
            thresholded,hand_segment=hand
            
            cv2.drawContours(img,[hand_segment+(ROI_right,ROI_top)],-1,(255,0,0),1)
            
            cv2.imshow("Threshold_image_of_hand",thresholded)
            
            thresholded=cv2.resize(thresholded,(200,200))
            
            
            cv2.imshow("real_image",thresholded)
            
            
            thresholded=np.reshape(thresholded,(1,thresholded.shape[0],thresholded.shape[1],1))
            
            pred=model1.predict(thresholded)
            #print(pred)
            if(np.argmax(pred) in word_dict):
                cv2.putText(img, word_dict[np.argmax(pred)],(170, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            else:
                cv2.putText(img, "NOT A SIGN",(170, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
            
    cv2.rectangle(img,(ROI_left,ROI_top),(ROI_right,ROI_bottom),(255,118,0),2)
            
    num_frames+=1
    
    cv2.putText(img, "hand sign recognition_ _ _",
    (10, 20), cv2.FONT_ITALIC, 0.5, (51,255,51), 1)
    
    cv2.imshow("Sign Detection", img)
    

    if(cv2.waitKey(25) & 0xFF==ord('q')):
        cv2.destroyAllWindows()
        break

camera.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 141ms/step


1/1 [==============================] - 0s 157ms/step


In [8]:
import cv2
import numpy as np

# Define the region of interest (ROI) coordinates
ROI_top = 100
ROI_bottom = 300
ROI_right = 150
ROI_left = 350

# Load the hand sign recognition model (model1)
# Replace 'model1' with your actual model object

# Load the word dictionary
word_dict={
    0: "A",
    1: "B",
    2: "C",
    3: "D",
    4: "E",
    5: "F",
    6: "G",
    7: "H",
    8: "I",
    9: "J",
    10: "K",
    11: "L",
    12: "M",
    13: "N",
    14: "O",
    15: "P",
    16: "Q",
    17: "R",
    18: "S",
    19: "T",
    20: "U",
    21: "V",
    22: "W",
    23: "X",
    24: "Y",
    25: "Z",
    26: "background"
}

camera = cv2.VideoCapture(0)
num_frames = 0

while True:
    _, image_frame = camera.read()
    
    image_frame = cv2.flip(image_frame, 1)
    
    img = image_frame.copy()
    
    roi = image_frame[ROI_top:ROI_bottom, ROI_right:ROI_left]
    
    img_grayscale = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    
    img_grayscale_blurr = cv2.GaussianBlur(img_grayscale, (9, 9), 0)
    
    if num_frames < 70:
        if num_frames == 0:
            accumulated_weight = np.float32(img_grayscale_blurr)
        
        cv2.accumulateWeighted(img_grayscale_blurr, accumulated_weight, 0.5)
        
        cv2.putText(img, "FETCHING BACKGROUND PLEASE WAIT", (80, 350), cv2.FONT_HERSHEY_SIMPLEX,
                    0.9, (0, 0, 255), 2)
    else:
        hand = hand_segmentation(img_grayscale_blurr)
        
        if hand is not None:
            thresholded, hand_segment = hand
            
            cv2.drawContours(img, [hand_segment + (ROI_right, ROI_top)], -1, (255, 0, 0), 1)
            
            cv2.imshow("Threshold_image_of_hand", thresholded)
            
            thresholded = cv2.resize(thresholded, (200, 200))
            
            cv2.imshow("real_image", thresholded)
            
            thresholded = np.reshape(thresholded, (1, thresholded.shape[0], thresholded.shape[1], 1))
            
            pred = model1.predict(thresholded)
            
            if np.argmax(pred) + 1 in word_dict:
                cv2.putText(img, word_dict[np.argmax(pred) + 1], (170, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                cv2.putText(img, "NOT A SIGN", (170, 45), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    
    cv2.rectangle(img, (ROI_left, ROI_top), (ROI_right, ROI_bottom), (255, 118, 0), 2)
    
    num_frames += 1
    cv2.putText(img, "hand sign recognition_ _ _",
                (10, 20), cv2.FONT_ITALIC, 0.5, (51, 255, 51), 1)
    
    cv2.imshow("Sign Detection", img)
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 157ms/step


1/1 [==============================] - 0s 151ms/step


1/1 [==============================] - 0s 126ms/step


1/1 [==============================] - 0s 141ms/step


1/1 [==============================] - 0s 157ms/step
